Interview Problem: Build Minesweeper
------

<center><img src="images/b62.png" width="700"/></center>

http://minesweeperonline.com/

How do you play Minesweeper?
------

Minesweeper is a game where the objective is correctly identify the location of all mines in a given grid. 

You are given a uniform grid of gray squares in the beginning of the game. 

Each square contains either a mine or an empty square. 

Empty squares have a number indicating the count of mines in the adjacent squares. 

Empty squares can have counts from zero (no adjacent mines) up to 8 (all adjacent squares are mines).

https://techdevguide.withgoogle.com/resources/coding-question-minesweeper/

https://www.geeksforgeeks.org/cpp-implementation-minesweeper-game/

What are the design elements?
----------

- Board
- Mines
- Current state of board
- Based on action, update board
- Scoring
- End game

What user stories do you want to support?
------

As a Minesweeper player, I would like to:

- start the game with a blank board
- pick a square to reveal
- __mark a potential mine with a flag__
- get an updated board
- either win or reveal the entire board
- play again

What functions do you need?
-------

- Primary functions
    - playgame / main function
    - parseinput
    - getnumbers (Separate actions from display, return vs print)
    - showcells 

What functions do you need?
-------

- Helper functions
    - setupgrid
    - showgrid
    - getrandomcell
    - getmines
    - getneighbors

Given that you are not a designer, how would you represent it?
-------

```
 a   b   c   d   e   f   g   h   i   
   -------------------------------------
 1 | 0 | 0 | 0 | 0 | 0 | 1 |   |   |   |
   -------------------------------------
 2 | 1 | 1 | 1 | 1 | 2 | 2 |   |   |   |
   -------------------------------------
 3 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 4 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 5 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 6 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 7 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 8 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 9 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
```

In [1]:
"""A command line version of Minesweeper"""
import random
import re
import time
from string import ascii_lowercase as letters


def setupgrid(gridsize, start, n_mines):
    emptygrid = [['0' for i in range(gridsize)] 
                  for i in range(gridsize)]

    mines = getmines(emptygrid, start, n_mines)

    for i, j in mines:
        emptygrid[i][j] = 'X'

    grid = getnumbers(emptygrid)

    return grid, mines


def showgrid(grid):
    gridsize = len(grid)

    horizontal = '   ' + (4 * gridsize * '-') + '-'

    # Print top column letters
    toplabel = '     '

    for i in letters[:gridsize]:
        toplabel = toplabel + i + '   '

    print(toplabel + '\n' + horizontal)

    # Print left row numbers
    for idx, i in enumerate(grid):
        row = '{0:2} |'.format(idx + 1)

        for j in i:
            row = row + ' ' + j + ' |'

        print(row + '\n' + horizontal)

    print('')


def getrandomcell(grid):
    gridsize = len(grid)

    a = random.randint(0, gridsize - 1)
    b = random.randint(0, gridsize - 1)

    return a, b


def getneighbors(grid, rowno, colno):
    gridsize = len(grid)
    neighbors = []

    for i in range(-1, 2):
        for j in range(-1, 2):
            if (i == 0) and (j == 0):
                continue
            elif (-1 < (rowno + i) < gridsize) and (-1 < (colno + j) < gridsize):
                neighbors.append((rowno + i, colno + j))

    return neighbors


def getmines(grid, start, n_mines):
    mines = []
    neighbors = getneighbors(grid, *start)

    for _ in range(n_mines):
        cell = getrandomcell(grid)
        while (cell == start or cell in mines) or (cell in neighbors):
            cell = getrandomcell(grid)
        mines.append(cell)

    return mines


def getnumbers(grid):
    for rowno, row in enumerate(grid):
        for colno, cell in enumerate(row):
            if cell != 'X':
                # Gets the values of the neighbors
                values = [grid[r][c] for r, c in getneighbors(grid, rowno, colno)]

                # Counts how many are mines
                grid[rowno][colno] = str(values.count('X'))

    return grid


def showcells(grid, currgrid, rowno, colno):
    # Exit function if the cell was already shown
    if currgrid[rowno][colno] != ' ':
        return

    # Show current cell
    currgrid[rowno][colno] = grid[rowno][colno]

    # Get the neighbors if the cell is empty
    if grid[rowno][colno] == '0':
        for r, c in getneighbors(grid, rowno, colno):
            # Repeat function for each neighbor that doesn't have a flag
            if currgrid[r][c] != 'F':
                showcells(grid, currgrid, r, c)


def playagain():
    choice = input('Play again? (y/n): ')

    return choice.lower() == 'y'


def parseinput(inputstring, gridsize, helpmessage):
    cell = ()
    flag = False
    message = "Invalid cell. " + helpmessage

    pattern = r'([a-{}])([0-9]+)(f?)'.format(letters[gridsize - 1])
    validinput = re.match(pattern, inputstring)

    if inputstring == 'help':
        message = helpmessage

    elif validinput:
        rowno = int(validinput.group(2)) - 1
        colno = letters.index(validinput.group(1))
        flag = bool(validinput.group(3))

        if -1 < rowno < gridsize:
            cell = (rowno, colno)
            message = ''

    return {'cell': cell, 'flag': flag, 'message': message}


def playgame():
    gridsize = 9
    numberofmines = 10

    currgrid = [[' ' for i in range(gridsize)] for i in range(gridsize)]

    grid = []
    flags = []
    starttime = 0

    helpmessage = ("Type the column followed by the row (eg. a5). "
                   "To put or remove a flag, add 'f' to the cell (eg. a5f).")

    showgrid(currgrid)
    print(helpmessage + " Type 'help' to show this message again.\n")

    while True:
        minesleft = numberofmines - len(flags)
        prompt = input('Enter the cell ({} mines left): '.format(minesleft))
        result = parseinput(prompt, gridsize, helpmessage + '\n')

        message = result['message']
        cell = result['cell']

        if cell:
            print('\n\n')
            rowno, colno = cell
            currcell = currgrid[rowno][colno]
            flag = result['flag']

            if not grid:
                grid, mines = setupgrid(gridsize, cell, numberofmines)
            if not starttime:
                starttime = time.time()

            if flag:
                # Add a flag if the cell is empty
                if currcell == ' ':
                    currgrid[rowno][colno] = 'F'
                    flags.append(cell)
                # Remove the flag if there is one
                elif currcell == 'F':
                    currgrid[rowno][colno] = ' '
                    flags.remove(cell)
                else:
                    message = 'Cannot put a flag there'

            # If there is a flag there, show a message
            elif cell in flags:
                message = 'There is a flag there'

            elif grid[rowno][colno] == 'X':
                print('Game Over\n')
                showgrid(grid)
                if playagain():
                    playgame()
                return

            elif currcell == ' ':
                showcells(grid, currgrid, rowno, colno)

            else:
                message = "That cell is already shown"

            if set(flags) == set(mines):
                minutes, seconds = divmod(int(time.time() - starttime), 60)
                print(
                    'You Win. '
                    'It took you {} minutes and {} seconds.\n'.format(minutes,
                                                                      seconds))
                showgrid(grid)
                if playagain():
                    playgame()
                return

        showgrid(currgrid)
        print(message)

playgame()

     a   b   c   d   e   f   g   h   i   
   -------------------------------------
 1 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 2 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 3 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 4 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 5 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 6 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 7 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 8 |   |   |   |   |   |   |   |   |   |
   -------------------------------------
 9 |   |   |   |   |   |   |   |   |   |
   -------------------------------------

Type the column followed by the row (eg. a5). To put or remove a flag, add 'f' to the cell (eg. a5f). Type 'help' to show this message again.

Enter the cell (10 mines left): a1


KeyboardInterrupt: 

[Source](https://gist.github.com/mohd-akram/3057736)

[A nice implementation in C](https://gist.github.com/dgossow/d28083522608771e1c65f49822820ba9)

[A nice implementation in Java](https://www.careercup.com/question?id=5633651648757760)

<br>
<br> 
<br>

----